# Dask yellow_tripdata csv to parquet converter

This notebook
* reads `yellow_tripdata_*.csv` (acquired using truncated https://github.com/toddwschneider/nyc-taxi-data/blob/master/setup_files/raw_data_urls.txt).
* writes data to parquet, with partitions of approximately 10MB. This is achieved by reading the data with a blocksize of 10MB, but could also be achieved by [repartitioning](https://docs.dask.org/en/latest/dataframe-best-practices.html#repartition-to-reduce-overhead) the dataframe before writing it.

This has been tested with data from 2014 only. A 10M partition size may be sub-optimal, but it helps to avoid `Worker exceeded 95% memory budget. Restarting` warnings.

%%capture
%%bash
pip install -U dask fsspec>=0.3.3 pyarrow
aws s3 cp s3://xdss-public-datasets/demos/taxi_1B.hdf5 datasets/taxi_1B.hdf5

In [1]:
import pandas as pd
from glob import glob
from dask.distributed import LocalCluster, Client, progress
from dask import dataframe as dd

In [2]:
# Blocksize used by dd.read_csv
blocksize='10M'

# data_path = '../datasets/taxi_1m.csv'
csv_data_path = '../../nyc-taxi-data/data/yellow_tripdata_*.csv'
parquet_data_path = f'../../data/yellow_trip_data_{blocksize}.parquet'

In [3]:
lc = LocalCluster(n_workers=6, threads_per_worker=2)
client = Client(lc)
client

Client Scheduler: tcp://127.0.0.1:36985 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 12 Memory: 33.69 GB


### Read a sample of the data

In [4]:
csv_files = glob(csv_data_path)

sample = pd.read_csv(csv_files[0],
                     skipinitialspace=True,
                     dtype={'store_and_fwd_flag': str},
                     parse_dates=['pickup_datetime', 'dropoff_datetime'],
                     nrows=100
                    )
sample.info()
sample.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   vendor_id           100 non-null    object        
 1   pickup_datetime     100 non-null    datetime64[ns]
 2   dropoff_datetime    100 non-null    datetime64[ns]
 3   passenger_count     100 non-null    int64         
 4   trip_distance       100 non-null    float64       
 5   pickup_longitude    100 non-null    float64       
 6   pickup_latitude     100 non-null    float64       
 7   rate_code           100 non-null    int64         
 8   store_and_fwd_flag  0 non-null      object        
 9   dropoff_longitude   100 non-null    float64       
 10  dropoff_latitude    100 non-null    float64       
 11  payment_type        100 non-null    object        
 12  fare_amount         100 non-null    float64       
 13  surcharge           100 non-null    float64       


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2014-05-31 17:17:00,2014-05-31 17:17:00,2,0.00,0.000000,0.000000,1,NaN,0.000000,0.000000,CSH,2.5,0.0,0.5,0.00,0.00,3.0
1,VTS,2014-05-31 17:07:00,2014-05-31 17:19:00,2,2.22,-73.987040,40.760120,1,NaN,-74.005550,40.745610,CSH,10.5,0.0,0.5,0.00,0.00,11.0
2,VTS,2014-05-31 16:29:00,2014-05-31 17:17:00,1,17.11,0.000000,0.000000,2,NaN,-73.982200,40.750920,CRD,52.0,0.0,0.5,11.47,5.33,69.3
3,VTS,2014-05-29 13:49:00,2014-05-29 14:03:00,1,0.82,-73.983312,40.734602,1,NaN,-73.995767,40.736920,CRD,9.5,0.0,0.5,1.90,0.00,11.9
4,VTS,2014-05-29 13:56:00,2014-05-29 14:06:00,1,0.86,-73.989925,40.745145,1,NaN,-73.985975,40.754945,CRD,8.0,0.0,0.5,1.60,0.00,10.1


### Get the sample data dtypes and datetime columns

In [5]:
dtypes = sample.select_dtypes(exclude=['datetime64[ns]']).dtypes.to_dict()
date_columns = sample.select_dtypes(include=['datetime64[ns]']).columns.to_list()

### Read the data from csv

In [6]:
df = dd.read_csv('../../nyc-taxi-data/data/yellow_tripdata_*.csv',
                 skipinitialspace=True,
                 dtype=dtypes,
                 parse_dates=date_columns,
                 blocksize=blocksize,
                )

df.info()
df.head()

<class 'dask.dataframe.core.DataFrame'>
Columns: 18 entries, vendor_id to total_amount
dtypes: datetime64[ns](2), object(3), float64(11), int64(2)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0.0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0.0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0.0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0.0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0.0,8.75


In [7]:
partition_size = df.get_partition(0).memory_usage().sum().compute()
print(f'DataFrame has {df.npartitions} partitions of size {partition_size}')

DataFrame has 2791 partitions of size 8276384


### Write the data to parquet

In [8]:
parquet_write = df\
.to_parquet(path=parquet_data_path,
            engine='pyarrow',
            compression='snappy',
            write_index=False,
            append=False,
            compute=False
           )\
.persist()

progress(parquet_write)

VBox()

%%capture
!aws s3 cp --recursive ../datasets/taxi_parquet s3://xdss-public-datasets/demos/taxi_parquet